# Implementing BB84 protocol with cirq

This notebook is based on [lecture notes from TU Delft university](https://ocw.tudelft.nl/wp-content/uploads/LN_Week6.pdf)

## What is Quantum Key Distribution?

### First, private key cryptography

Imagine you've written a secret note that you only want your best friend to read. To keep it secret, you decide to scramble the letters in a specific way, a way that only you and your friend understand. This method of scrambling is your _key_. Now, even if someone else gets the note, they won't understand what it says because it's all scrambled up. Your friend, who knows the key, can unscramble the note to read the original message. This is the basic idea behind the private key, or symmetric, cryptography.

Private key cryptography is a type of encryption where the same key is used to both encrypt and decrypt the message. You use the key (a specific set of instructions) to convert your message into a secret code, and the same key is used to convert the secret code back into the original message. Because the same key is used for both processes, it's really important to keep the key secret. If someone else gets the key, they can decode all of your messages.

The main challenge in private key cryptography is securely sharing the key with the person you want to communicate with. If you're not careful, someone else might intercept the key and decode your messages. This is known as the _key distribution problem_.

### Quantum Key Distribution to the rescue!

Quantum Key Distribution (QKD) offers a solution to this problem using the principles of quantum mechanics. In the quantum world, measuring a system can change its state. This is called the _observer effect._ If someone tries to intercept the key and measures these photons, they will inevitably change their states. Your best friend will notice these changes when he receives the qubits, so they will know there was an eavesdropper. QKD solves the key distribution problem by enabling you and your friend to detect eavesdropping. If there's no eavesdropping, you can use the sequence of qubits that you've sent and your friend received to create a secret key. If there is eavesdropping, you discard the key and try again.

BB84 is one of the protocols realizing idea of QKD.

## Our battle plan and idea

In here we will not dive deep into mathematics (physics) behind every aspect of the protocol. I will provide you with references if you want to dive deeper. **Within scope of this notebook we are focusd on building the BB84 protocol using cirq, while allowing our solution not to be prefect, because we are learning.** 

With that preface our plan looks like this:
 1. Implement noise-free BB84 of the protocol
 2. Add some eavesdropping!
 3. Check how secure is the protocol.
 4. Add noise to the BB84 protocol.
 5. Double check how secure is the protocol.
 6. Use it to send secret message that is protected by law of physics.

## 1. Noise-free BB84

For the sake of the document sender will be called Alice and receiver will be called Bob. 

To implement noise-free BB84 we will follow this steps:
 1. Alice will generate two random binary (consisting only of `0`s and `1`s) strings - `alice_a`, `alice_b` - of the same length.
 2. Alice will use her strings to prepare a quantum state (sequence of qubits) that she will send to Bob.
 3. Bob will generate random binary string `bob_b` that he will use to try to _guess_ and decode Alice's message. Then Bob tells Alice that he received and measured the state he received.
 4. Alice and Bob exchange their `_b` strings. They discard all bits that do not match (from Alice's original message and Bob's guess-decoded message). They are left with `alice_kept_a` and `bob_kept_a` strings, respectively. 
 5. Alice picks a random set of indices of `alice_kept_a` - `test_indices` and sends them to Bob. Alice and Bob use `test_indices` to create binary strings `alice_test_a` and `bob_test_a`. **If any of the bits miss-match they restart the procedure**
 6. They now use remaining indeces (without `test_indices`) - `final_indices` - and treat is as seed for key creation via the process of privay amplification (we will discuss it later, do not worry!)
 
Let's get to it!

### Step 0

Import cirq and let's declare cirq's simulator (it enables quantum simulation!).

In [18]:
import cirq
import numpy as np

simulator = cirq.Simulator()

### Step 1
Our first step is to generate two random binary strings (represented as arrays of `0` and `1`s) - `alice_a` and `alice_b`. In the [previous screencast](https://www.youtube.com/watch?v=kANpuN2Ax0A) we build quantum random number generator. We will leverage this solution right now!

We need to of course decide on the length of our initial strings. We will use length that is a multiple of 4, so for `n` the length will be `4n`. On "average" in step 4 half of the bits would missmatch leaving us with `2n`. In step 6, the `final_indices` will again be of half-length, leaving us with `n`.

In [33]:
from quantum_lib.qrng import QuantumRandomNumberGenerator

qrng = QuantumRandomNumberGenerator(simulator)

n = 4
string_length = 4 * n

alice_a = qrng.generate_binary_array(string_length)
alice_b = qrng.generate_binary_array(string_length)
print(alice_a)
print(alice_b)

[1 1 0 0 1 0 1 1 1 0 1 1 0 0 0 0]
[0 0 0 0 0 0 1 1 1 1 1 0 0 1 1 1]


### Step 2
Now it is time to prepare a quantum state that Alice will send to Bob. In our simplified version we will use good old H-gate. The idea is simple, if the `alice_b[i] == 1` we will apply H-gate to that particular qubit. For sciency folks - we are changing _computational basis_.

To do that we will first need to create an array of qubits of `string_length`. Recall that in cirq all qubits are in "0" ($| 0 \rangle$) state initially. In classical computing we would use the _NOT_ gate. It's quantum analogue is called _X-gate_. Summing up our tactics looks like this:
 1. if `alice_a[idx] == 1` then apply `X` gate.
 2. If `alice_b[idx] == 1` then apply `H` gate.

In [48]:
quantum_channel = cirq.Circuit()
channel_qubits = cirq.NamedQubit.range(string_length, prefix="q_")
for idx in range(string_length):
    if alice_a[idx] == 1:
        quantum_channel.append(cirq.X(channel_qubits[idx]))
    if alice_b[idx] == 1:
        quantum_channel.append(cirq.H(channel_qubits[idx]))
print(quantum_channel)

q_0: ────X───────

q_1: ────X───────

q_4: ────X───────

q_6: ────X───H───

q_7: ────X───H───

q_8: ────X───H───

q_9: ────H───────

q_10: ───X───H───

q_11: ───X───────

q_13: ───H───────

q_14: ───H───────

q_15: ───H───────


We can see that the qubits that did not need either `X` or `H` gate are not showing up. Do not worry they _will_ show up when we will measure the state! We called the variable `quantum_channel` to underscore the fact that on one side we have Alice preparing the qubits (that is - state) and on the other hand we will have Bob reading the state (that is - the qubits).

### Step 3
In this step Bob will try to "guess-decode" the `alice_a` string. How? He will generate his own `bob_b` string and whenever `bob_b[idx] == 1` he will "revert" the `H` gate. How? `H` gate has this awesome property that if you apply it twice you get your original state!

In [47]:
bob_b = qrng.generate_binary_array(string_length)
for idx in range(string_length):
    if bob_b[idx] == 1:
        quantum_channel.append(cirq.H(channel_qubits[idx]))
quantum_channel.append(cirq.measure(channel_qubits, key="bob_measurement"))

In [42]:
print(quantum_channel)

q_0: ────X───────────M('bob_measurement')───
                     │
q_1: ────X───H───────M──────────────────────
                     │
q_2: ────H───────────M──────────────────────
                     │
q_3: ────────────────M──────────────────────
                     │
q_4: ────X───────────M──────────────────────
                     │
q_5: ────────────────M──────────────────────
                     │
q_6: ────X───H───────M──────────────────────
                     │
q_7: ────X───H───H───M──────────────────────
                     │
q_8: ────X───H───H───M──────────────────────
                     │
q_9: ────H───H───────M──────────────────────
                     │
q_10: ───X───H───────M──────────────────────
                     │
q_11: ───X───────────M──────────────────────
                     │
q_12: ───────────────M──────────────────────
                     │
q_13: ───H───H───────M──────────────────────
                     │
q_14: ───H───H───────M──────────────────────
   

In [3]:
### What is privacy amplification?

In [ ]:
### What about Eve?

## Let's make some noise!

## Time to send a secret message